### 01. 라이브러리 및 데이터 호출

In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras import backend as K

In [36]:
train = pd.read_csv('data/train.csv')
submission = pd.read_csv('data/submission.csv')

--------------------------

### 02. 데이터 기초 정보 확인

In [37]:
# 데이터 처음,끝부분 확인
head = train.head(5)
tail = train.head(5)

# Columns 확인
columns = train.columns

# Data Shape 확인
shape = train.shape

# 통계값 확인
describe = train.describe()

# Columns Type 확인
dtypes = train.dtypes

-----------------------

### 03. 데이터 전처리

#### 3-1. 결측치 확인
###### [1] 결측치와 중복되는 행이 모두 없음을 확인할 수 있다
###### [2] 따라서 결측치 처리에 대한 부분이 불필요

In [38]:
# 컬럼별 결측치 파악
null = train.isna().sum()

# 중복되는 행 파악
duplicated = train[train.duplicated()]

#### 3-2. 시간 데이터 전처리
###### [1] Time 열을 datetime 형식으로 변환
###### [2] Time 열을 DataFrame의 인덱스로 설정

In [39]:
# Time 열을 datetime 형식으로 변환
train['Time'] = pd.to_datetime(train['Time'])
submission['Time'] = pd.to_datetime(submission['Time'])

# 데이터 정렬
train.sort_values('Time', inplace=True)

#### 3-3. 데이터 스케일링

In [40]:
# MinMaxScaler 객체를 생성 (이 객체는 데이터를 0과 1 사이로 변환)
scaler = MinMaxScaler(feature_range=(0, 1))

# train 데이터의 모든 컬럼에 대해 스케일링 적용
scaled_data = scaler.fit_transform(train[['Open', 'High', 'Low', 'Close', 'Volume', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av']])

-------------------------

#### 4. 데이터 모델링

##### 4-1. LSTM 입력 형식에 맞게 데이터 재구성
###### [1] 

In [41]:
# LSTM 입력 형식에 맞게 데이터 재구성
sequence_length = 60
X_train, y_train = [], []
for i in range(sequence_length, len(scaled_data)):
    X_train.append(scaled_data[i-sequence_length:i])
    y_train.append(scaled_data[i, 3])  # Close 값 예측

X_train, y_train = np.array(X_train), np.array(y_train)

##### 4-2. LSTM 모델 구성

In [42]:
# LSTM 모델 구성
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

c:\Users\NT930QCG\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


##### 4-3. 모델 컴파일 및 학습

In [43]:
# RMSE 손실 함수 정의
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# 모델 컴파일 (손실 함수로 RMSE 사용)
model.compile(optimizer='adam', loss=rmse)

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - loss: 0.0391
Epoch 2/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 89s 52ms/step - loss: 0.0214
Epoch 3/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 121s 71ms/step - loss: 0.0205
Epoch 4/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 146s 86ms/step - loss: 0.0201
Epoch 5/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 69672s 41s/step - loss: 0.0200
Epoch 6/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 88s 52ms/step - loss: 0.0202
Epoch 7/20
1419/1704 ━━━━━━━━━━━━━━━━━━━━ 23s 82ms/step - loss: 0.0197

##### 4-4. 모델 예측

In [ ]:
# 예측 준비
last_sequence = scaled_data[-sequence_length:]  # 마지막 시퀀스
predicted_closes = []

for i in range(len(submission)):
    input_data = np.expand_dims(last_sequence, axis=0)
    predicted_close = model.predict(input_data)
    predicted_closes.append(predicted_close[0, 0])

    # 예측된 값을 마지막 시퀀스에 추가하고 스케일링 해제
    new_sequence = np.concatenate((last_sequence[1:], np.array([[0, 0, 0, predicted_close, 0, 0, 0, 0]])), axis=0)
    last_sequence = new_sequence

##### 4-5. 스케일링 되돌리기

In [ ]:
# 스케일링 되돌리기
predicted_closes = scaler.inverse_transform(np.concatenate((np.zeros((len(predicted_closes), 3)), np.array(predicted_closes).reshape(-1, 1), np.zeros((len(predicted_closes), 4))), axis=1))[:, 3]

#### 5. 제출 파일 생성

In [ ]:
# 스케일링 되돌리기
predicted_closes = scaler.inverse_transform(np.concatenate((np.zeros((len(predicted_closes), 3)), np.array(predicted_closes).reshape(-1, 1), np.zeros((len(predicted_closes), 4))), axis=1))[:, 3]

# 제출 파일 생성
submission['Close'] = predicted_closes
submission.to_csv('data/submission_result.csv', index=False)
print("Submission file created successfully.")